# Импорты

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import control 
import sympy
import os
import scipy
import cvxpy 
import sympy.plotting
import array_to_latex as a2l

In [2]:
np.set_printoptions(precision=2)
sympy.init_printing()
p = sympy.Symbol("p")
s = sympy.Symbol("s")
t = sympy.Symbol("t")
w = sympy.Symbol("w")
I = sympy.I

In [3]:
def get_t(end_t = 10, dt=0.001, start_t = 0):
    return np.linspace(start_t, end_t, int(end_t / dt))

In [4]:
def get_controllability_matrix(A, B):
    ctrb_m = np.hstack((B, *[(np.linalg.matrix_power(A, i)) @ B for i in range(1, A.shape[0])]))
    assert np.allclose(control.ctrb(A, B), ctrb_m), 'Smth wrong'
    return ctrb_m

def get_observability_matrix(A, C):
    obsv_m = np.vstack((C, *[C @ np.linalg.matrix_power(A, i) for i in range(1, A.shape[0])]))
    assert np.allclose(control.obsv(A, C), obsv_m), 'Smth wrong'
    return obsv_m
        
def check_controllability_eigens(A, B):
    eig_vals = np.linalg.eigvals(A)
    print(f'Eigen values of A:')
    for val in eig_vals:
        print(f"   {np.array([val])}: {'controllable' if np.linalg.matrix_rank(np.hstack(((A - val * np.eye(A.shape[0])), B))) == A.shape[0] else 'not controllable'}")

def check_observability_eigens(C, A):
    eig_vals = np.linalg.eigvals(A)
    print(f'Eigen values of A:')
    for val in eig_vals:
        print(f"   {np.array([val])}: {'observable' if np.linalg.matrix_rank(np.vstack(((A - val * np.eye(A.shape[0])), C))) == A.shape[0] else 'not observable'}")


# Задание 1

In [5]:
A = np.array([
    [0, 1],
    [0, 0]
])

B_1 = np.array([
    [1, 1, 0],
    [0, 1, 0]
])

B_2 = np.array([[0],
                [1]])

C_1 = np.array([[1, 0]])
D_1 = np.array([[0, 0, 1]])

In [6]:
task1_C_2s = np.array([
    [[1, 0],
     [0, 1],
     [0, 0]],
    [[1, 1],
     [0, 2],
     [0, 0]],
])
task1_D_2s = np.array([[[0], [0], [1]], [[0], [0], [2]]])

ts = get_t(15)
w = np.vstack([0.05 * np.sin(ts), 0.01 * np.sin(10 * ts), 0.01 * np.sin(10 * ts)])

In [7]:
omega_i = sympy.Symbol("omega",real=True) * sympy.I

def get_fraction(tf):
    num, den = tf.num[0][0], tf.den[0][0]
    den_ = sum((0 if abs(co) < 1e-3 else co) * omega_i**i for i, co in enumerate(reversed(den)))
    num_ = sum((0 if abs(co) < 1e-3 else co) * omega_i**i for i, co in enumerate(reversed(num)))
    return num_ / den_

In [8]:
for i in range(2):
    print('\n______________________________')
    task1_C_2 = task1_C_2s[i]
    task1_D_2 = task1_D_2s[i]
    check_controllability_eigens(A, B_2)
    check_observability_eigens(task1_C_2, A)
    Q = task1_C_2.T @ task1_C_2
    R = task1_D_2.T @ task1_D_2
    K, S, E = control.lqr(A, B_2, Q, R)
    print(f'\[C_2 = {a2l.to_ltx(task1_C_2, print_out=False)}; D_2 = {a2l.to_ltx(task1_D_2, print_out=False)};\]')
    print(f'\[C_2^T D_2 = 0: {np.all(task1_C_2.T @ task1_D_2 == 0)}\]')
    print(f'\[D_2^T D_2 \\text{"{ обратима}"}: {np.linalg.det(task1_D_2.T @ task1_D_2) != 0}\]')
    print(f'\[spec(A-B_2 K) = {a2l.to_ltx(E, print_out=False)}\]')
    print(f'\[Q = {a2l.to_ltx(S, print_out=False)}\]')
    print(f'\[K = {a2l.to_ltx(K, print_out=False)}\]')

    ss = control.ss(A - B_2@K, B_1, task1_C_2 - task1_D_2 @ K, np.zeros((task1_C_2.shape[0], B_1.shape[1])))
    tf = control.ss2tf(ss)
    
    smatrix = []
    for row in range(tf.noutputs):
        srow = []
        for col in range(tf.ninputs):
            srow.append(get_fraction(tf[row, col]))
        smatrix.append(srow)
    smatrix = sympy.Matrix(smatrix)
    sympy.print_latex(smatrix)
    
    gram_obs = control.gram(ss, "o")
    print(f'\[||W||_{"{H_2}"} = {np.sqrt(np.trace(B_1.T @ gram_obs @ B_1))}\]')

    # Simulation
    resp = control.forced_response(ss, X0=np.ones((2, 1)), T=ts, U=w)
    for indx, z in enumerate(resp.outputs):
        plt.plot(ts, z, label=f'$z_{indx}$')
    plt.xlabel('t, c')
    plt.ylabel('z')
    plt.legend()
    plt.close()

    # Frequency response
    for ni in range(task1_C_2.shape[0]):
        for nj in range(B_1.shape[1]):
            mag, phase, omega = control.bode(tf[ni, nj], omega=np.arange(10**-3, 10**3, 10**-2), plot=False)
            plt.plot(omega, mag)
    plt.yscale('log')
    plt.xscale('log')
    plt.xlabel('w, rad/s')
    plt.ylabel('Amp')
    plt.close()
    
    # Singular values plot
    sigma, omega = control.singular_values_plot(ss, plot=False)
    for s in sigma:
        plt.plot(omega, s)
    plt.grid()
    plt.xlabel('$\omega, рад/с$')
    plt.ylabel('$\sigma$')
    plt.close()
    
    print(f'\[||W||_H_\\{"infty"} = {sigma.max()} \]')
    


______________________________
Eigen values of A:
   [0.]: controllable
   [0.]: controllable
Eigen values of A:
   [0.]: observable
   [0.]: observable
\[C_2 = \begin{bmatrix}
  1.00 &  0.00\\
  0.00 &  1.00\\
  0.00 &  0.00
\end{bmatrix}; D_2 = \begin{bmatrix}
  0.00\\
  0.00\\
  1.00
\end{bmatrix};\]
\[C_2^T D_2 = 0: True\]
\[D_2^T D_2 \text{ обратима}: True\]
\[spec(A-B_2 K) = \begin{bmatrix}
 -0.87 + 0.50j & -0.87 + -0.50j
\end{bmatrix}\]
\[Q = \begin{bmatrix}
  1.73 &  1.00\\
  1.00 &  1.73
\end{bmatrix}\]
\[K = \begin{bmatrix}
  1.00 &  1.73
\end{bmatrix}\]


ControlMIMONotImplemented: Not implemented for MIMO systems without slycot.

# Задание 2

In [ ]:
def generate_H2_obs(a, b_1, c_1, d_1):
    p = scipy.linalg.solve_continuous_are(a.T, c_1.T, b_1@b_1.T, d_1@d_1.T)
    return -p @ c_1.T @np.linalg.inv(d_1 @ d_1.T)

In [ ]:
def generate_H2_obs(a, b_1, c_1, d_1):
    p = scipy.linalg.solve_continuous_are(a.T, c_1.T, b_1 @ b_1.T, d_1 @ d_1.T)
    return -p @ c_1.T @ np.linalg.inv(d_1 @ d_1.T)


for i in range(2):
    print('\n______________________________')
    task1_C_2 = task1_C_2s[i]
    task1_D_2 = task1_D_2s[i]
    # check_controllability_eigens(A, B_2)
    # check_observability_eigens(task1_C_2, A)
    Q = task1_C_2.T @ task1_C_2
    R = task1_D_2.T @ task1_D_2
    K, S, E = control.lqr(A, B_2, Q, R)
    K = -K
    print(f'\[C_2 = {a2l.to_ltx(task1_C_2, print_out=False)}; D_2 = {a2l.to_ltx(task1_D_2, print_out=False)};\]')
    # print(f'\[C_2^T D_2 = 0: {np.all(task1_C_2.T @ task1_D_2 == 0)}\]')
    # print(f'\[D_2^T D_2 \\text{"{ обратима}"}: {np.linalg.det(task1_D_2.T @ task1_D_2) != 0}\]')
    # print(f'\[spec(A-B_2 K) = {a2l.to_ltx(E, print_out=False)}\]')
    # print(f'\[Q = {a2l.to_ltx(S, print_out=False)}\]')
    print(f'\[K = {a2l.to_ltx(K, print_out=False)}\]')

    L = generate_H2_obs(A, B_1, C_1, D_1)
    print(f'\[L = {a2l.to_ltx(L, print_out=False)}\]')

    new_A = np.block([[A, B_2 @ K], [-L @ C_1, A + B_2 @ K + L @ C_1]])
    new_B = np.block([[B_1], [-L @ D_1]])
    new_C = np.block([task1_C_2, -task1_D_2 @ K])
    ss = control.ss(new_A, new_B, new_C, 0)
    tf = control.ss2tf(ss)

    smatrix = []
    for row in range(tf.noutputs):
        srow = []
        for col in range(tf.ninputs):
            srow.append(get_fraction(tf[row, col]))
        smatrix.append(srow)
    smatrix = sympy.Matrix(smatrix)
    sympy.print_latex(smatrix)

    gram_obs = control.gram(ss, "o")
    print(f'\[||W||_{"{H_2}"} = {np.sqrt(np.trace(new_B.T @ gram_obs @ new_B))}\]')

    # Simulation
    resp = control.forced_response(ss, X0=[1, 2, 3, 4], T=ts, U=w)
    for indx, z in enumerate(resp.outputs):
        plt.plot(ts, z, label=f'$z_{indx}$')
    plt.xlabel('t, c')
    plt.ylabel('z')
    plt.legend()
    plt.close()

    # Frequency response
    for ni in range(task1_C_2.shape[0]):
        for nj in range(B_1.shape[1]):
            mag, phase, omega = control.bode(tf[ni, nj], omega=np.arange(10 ** -3, 10 ** 3, 10 ** -2), plot=False)
            plt.plot(omega, mag)
    plt.yscale('log')
    plt.xscale('log')
    plt.xlabel('w, rad/s')
    plt.ylabel('Amp')
    plt.close()

    # Singular values plot
    sigma, omega = control.singular_values_plot(ss, plot=False)
    for s in sigma:
        plt.plot(omega, s)
    plt.grid()
    plt.xlabel('$\omega, рад/с$')
    plt.ylabel('$\sigma$')
    plt.close()

    print(f'\[||W||_H_\\{"infty"} = {sigma.max()} \]')



# Задание 3

In [ ]:
C_2 = task1_C_2s[0]
D_2 = task1_D_2s[0]

In [ ]:
def generate_Hinf(a, b_2, c_2, d_2, b_1, gamma):
    R = b_2@np.linalg.inv(d_2.T@d_2)@b_2.T-(gamma**-2)*b_1@b_1.T
    q = scipy.linalg.solve_continuous_are(a,np.identity(R.shape[0]),c_2.T@c_2,np.linalg.inv(R))
    return -np.linalg.inv(d_2.T@d_2)@b_2.T@q

In [ ]:
gammas = [1.4, 2, 10]

In [ ]:
for i in range(3):
    print('\n\subsubsubsection{gamma = ' + str(gammas[i]) + '}')
    # check_controllability_eigens(A, B_2)
    # check_observability_eigens(C_2, A)
    Q = C_2.T @ C_2
    R = D_2.T @ D_2
    K = -generate_Hinf(A, B_2, C_2, D_2, B_1, gammas[i])
    # print(f'\[C_2 = {a2l.to_ltx(C_2, print_out=False)}; D_2 = {a2l.to_ltx(D_2, print_out=False)};\]')
    # print(f'\[C_2^T D_2 = 0: {np.all(C_2.T @ D_2 == 0)}\]')
    # print(f'\[D_2^T D_2 \\text{"{ обратима}"}: {np.linalg.det(D_2.T @ D_2) != 0}\]')
    print(f'\[spec(A-B_2 K) = {np.linalg.eigvals(A - B_2 @ K)}\]')
    # print(f'\[Q = {a2l.to_ltx(S, print_out=False)}\]')
    print(f'\[K = {a2l.to_ltx(K, print_out=False)}\]')

    ss = control.ss(A - B_2@K, B_1, C_2 - D_2 @ K, np.zeros((C_2.shape[0], B_1.shape[1])))
    tf = control.ss2tf(ss)
    
    smatrix = []
    for row in range(tf.noutputs):
        srow = []
        for col in range(tf.ninputs):
            srow.append(get_fraction(tf[row, col]))
        smatrix.append(srow)
    smatrix = sympy.Matrix(smatrix)
    sympy.print_latex(smatrix)
    
    gram_obs = control.gram(ss, "o")
    print(f'\[||W||_{"{H_2}"} = {np.sqrt(np.trace(B_1.T @ gram_obs @ B_1))}\]')

    # Simulation
    resp = control.forced_response(ss, X0=np.ones((2, 1)), T=ts, U=w)
    for indx, z in enumerate(resp.outputs):
        plt.plot(ts, z, label=f'$z_{indx}$')
    plt.xlabel('t, c')
    plt.ylabel('z')
    plt.legend()
    plt.close()

    # Frequency response
    for ni in range(C_2.shape[0]):
        for nj in range(B_1.shape[1]):
            mag, phase, omega = control.bode(tf[ni, nj], omega=np.arange(10**-3, 10**3, 10**-2), plot=False)
            plt.plot(omega, mag)
    plt.yscale('log')
    plt.xscale('log')
    plt.xlabel('w, rad/s')
    plt.ylabel('Amp')
    plt.close()
    
    # Singular values plot
    sigma, omega = control.singular_values_plot(ss, plot=False)
    for s in sigma:
        plt.plot(omega, s)
    plt.grid()
    plt.xlabel('$\omega, рад/с$')
    plt.ylabel('$\sigma$')
    plt.close()
    
    print(f'\[||W||_H_\\{"infty"} = {sigma.max()} \]')
    

# Задание 4

In [ ]:
def generate_Hinf_obs(a, b_1, b_2, c_1, c_2, d_1, d_2, gamma):
    R_1 = c_1.T @ np.linalg.inv(d_1 @ d_1.T) @c_1 - (gamma**-2) * c_2.T @ c_2
    R_2 = b_2 @ np.linalg.inv(d_2.T @ d_2) @ b_2.T - (gamma**-2) * b_1 @ b_1.T
    p = scipy.linalg.solve_continuous_are(a.T, np.identity(R_1.shape[0]), b_1@b_1.T, np.linalg.inv(R_1))
    q = scipy.linalg.solve_continuous_are(a, np.identity(R_2.shape[0]), c_2.T@c_2, np.linalg.inv(R_2))
    if np.max(np.linalg.eig(p@q)[0]) < gamma ** 2:
        l = -p@np.linalg.inv(np.identity(q.shape[0])-(gamma**-2)*q@p)@(c_1+(gamma**-2)*d_1@b_1.T@q).T@np.linalg.inv(d_1@d_1.T)
        k = -np.linalg.inv(d_2.T@d_2)@b_2.T@q
        return k, l, q
    return None

In [ ]:
for gamma in gammas:
    K_4_1, L_4_1, Q_1 = generate_Hinf_obs(A, B_1, B_2, C_1, C_2, D_1, D_2, gamma)
    print(K_4_1)

In [ ]:
for i in range(3):
    print('\n\subsubsubsection{gamma = ' + str(gammas[i]) + '}')
    K, L, Q = generate_Hinf_obs(A, B_1, B_2, C_1, C_2, D_1, D_2, gamma)
    print(f'\[spec(A-B_2 K) = {np.linalg.eigvals(A - B_2 @ K)}\]')
    print(f'\[K = {a2l.to_ltx(K, print_out=False)}\]')
    print(f'\[Q = {a2l.to_ltx(Q, print_out=False)}\]')
    print(f'\[L = {a2l.to_ltx(L, print_out=False)}\]')
    
    
    A_new = np.block([
    [A + B_2@K, -B_2@K],
    [-(L@D_1+B_1)*(10**-2)@B_1.T@Q, A + L@C_1 + (L@D_1+B_1)*(10**-2)@B_1.T@Q]
    ]) 
    B_new = np.block([
        [B_1],
        [L@D_1+B_1]
    ]) 
    C_new = np.block([C_2+D_2@K, -D_2@K])
    D_new = np.zeros((C_2.shape[0],D_1.shape[1]))    

    ss = control.ss(A_new,B_new,C_new,D_new)
    tf = control.ss2tf(ss)
    
    smatrix = []
    for row in range(tf.noutputs):
        srow = []
        for col in range(tf.ninputs):
            srow.append(get_fraction(tf[row, col]))
        smatrix.append(srow)
    smatrix = sympy.Matrix(smatrix)
    sympy.print_latex(smatrix)
    
    gram_obs = control.gram(ss, "o")
    print(f'\[||W||_{"{H_2}"} = {np.sqrt(np.trace(B_new.T @ gram_obs @ B_new))}\]')

    # Simulation
    resp = control.forced_response(ss, X0=[1, 2, 3, 4], T=ts, U=w)
    for indx, z in enumerate(resp.outputs):
        plt.plot(ts, z, label=f'$z_{indx}$')
    plt.xlabel('t, c')
    plt.ylabel('z')
    plt.legend()
    plt.close()

    # Frequency response
    for ni in range(C_2.shape[0]):
        for nj in range(B_1.shape[1]):
            mag, phase, omega = control.bode(tf[ni, nj], omega=np.arange(10**-3, 10**3, 10**-2), plot=False)
            plt.plot(omega, mag)
    plt.yscale('log')
    plt.xscale('log')
    plt.xlabel('w, rad/s')
    plt.ylabel('Amp')
    plt.close()
    
    # Singular values plot
    sigma, omega = control.singular_values_plot(ss, plot=False)
    for s in sigma:
        plt.plot(omega, s)
    plt.grid()
    plt.xlabel('$\omega, рад/с$')
    plt.ylabel('$\sigma$')
    plt.close()
    
    print(f'\[||W||_H_\\{"infty"} = {sigma.max()} \]')
    